# Satellite Image Classification

## Importing dependencies and setting file paths

In [ ]:

import random
import os
import glob
import cv2 
from fastai.vision import *
from fastai import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import plot_confusion_matrix
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')


In [ ]:
# Set seed fol all
def seed_everything(seed=1358):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
PATH = Path('../input/planets-dataset/planet/planet/')
train_img = PATH/'train-jpg'
train_folder = 'train-jpg'
test_img = PATH/'test-jpg'
model_dir = Path('/kaggle/working/')
bs = 64

In [ ]:
PATH.ls()

## Data Analysis

In [ ]:
train_df = pd.read_csv(os.path.join(PATH, 'train_classes.csv'))
# adding path to the image in our dataframe. 
train_df['image_name'] = train_df['image_name'].apply(lambda x: f'{train_folder}/{x}.jpg')
train_df.head()

> We have 17 unique labels for this data

In [ ]:
# Since this is a multi lable task and the labels are given as tags in a single dataframe series
biner = MultiLabelBinarizer()
#getting lables from data frame
tags = train_df['tags'].str.split()
#print(tags)

#numpy array of tags in binary form
y = biner.fit_transform(tags)
#print(y)
#print(type(y))
#print(y.shape)
#print()
labels = biner.classes_
print('Number of labels: ', len(labels))
print(labels)


In [ ]:
print(labels[0])

In [ ]:
#print(train_df['tags'].apply(lambda x: 1 if 'haze' in x.split()  else 0))

In [ ]:
#Getting the labels into one hot encoded form
#adds class lables as columns
for label in labels:
    train_df[label] = train_df['tags'].apply(lambda x: 1 if label in x.split()  else 0)
    
train_df.head()
#train_df.shape

The label primary appears the most in our dataset followed by clear and agriculture. 
As stated in the data description, primary refers to primary rainforest.
> Generally speaking, the "primary" label was used for any area that exhibited dense tree cover. 

In [ ]:
print(train_df[labels])

In [ ]:
print(train_df[labels].sum().sort_values(ascending=False))

In [ ]:
train_df[labels].sum().sort_values(ascending=True).plot(kind='barh', figsize=(8,8))

Looking at the co-ocurrance for these labels. 
> The combination (primary, clear) has the highest co-ocurrance. Followed by (primary, agriculture)

In [ ]:
df_asint = train_df.drop(train_df.columns[[0,1]], axis=1).astype(int)
coocc_df = df_asint.T.dot(df_asint)

coocc_df

In [ ]:
# Confusion matrix. 

Plotting a few random images with there labels to see how the data looks. 
Choose 10 random images from the data. 

In [ ]:
#reading images

random_imgs = train_df.ix[random.sample(list(train_df.index), 10)][['image_name', 'tags']]

to_read = random_imgs.loc[:, 'image_name'].values
tags = random_imgs.loc[:, 'tags'].values

images = [cv2.imread(os.path.join(PATH/file)) for file in to_read]
print("Number of images: ", len(images))
print("Size of an image: ", images[0].shape)

In [ ]:
to_read

In [ ]:
tags

In [ ]:
#print(images)

In [ ]:
plt.figure(figsize=(25,15))
columns = 5
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)
    plt.grid(False)
    plt.title(tags[i])

## Training 

Using fastai to train and evaluate models. 

In [ ]:
print(f"Size of Training set images: {len(list(train_img.glob('*.jpg')))}")
print(f"Size of Test set images: {len(list(test_img.glob('*.jpg')))}")


Starting with an image size of 128*128 with a few transformations. 
+ Flipping the image vertically and horizontaly. 
+ Changing lighting and contrast. 
+ Rotations. 
+ Zooming. 

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=True,p_lighting=0.4,
                      max_lighting=0.3, max_zoom=1.05, max_rotate=360, xtra_tfms=[flip_lr()])

In [ ]:
img_size = 128

#data augmentation
tfms = get_transforms(do_flip=True,flip_vert=True,p_lighting=0.4,
                      max_lighting=0.3, max_zoom=1.05, max_rotate=360, xtra_tfms=[flip_lr()])


# The datablock API
# using 2% of the training data to validate the models. 

src = (ImageList.from_df(train_df, PATH, cols='image_name')
        .split_by_rand_pct(valid_pct=0.2)
        .label_from_df(label_delim=' '))


data = (src.transform(tfms,size=img_size,resize_method=ResizeMethod.CROP)
        .databunch(bs=bs,num_workers=4) 
        .normalize(imagenet_stats)      
       )

98% images for training and 2% for validating. 

In [ ]:
data

Train data has 32384 images of size 128x128x3. 

In [ ]:
data.train_ds

In [ ]:
data.valid_ds

Picking a point from the train_ds from our databunch gives the Image object which includes its size. 
It also outputs that the label in Multicategory coupled with its value. 

In [ ]:
data.train_ds[0]

Selecting a random point from the databunch. 
The databunch containing the training dataset has both x and y components and we can also index them. 
This time selecting the labels from **data.train_ds.y** and image from **data.train_ds.x**



In [ ]:
print(data.train_ds.y[200])
data.train_ds.x[200]

In [ ]:
data.show_batch(rows=3, figsize=(12, 9))

+ 1st stage: Freezing early layers and only fine-tuning the last few newly added layers.
+ 2nd stage: Unfreezing all the layers and fine-tuning them. 


Using the learning rate finder before every stage. 


## Experiment 1

> For the first experiment: 
> + Using VGG16 (pretrained on imagenet)
> + fbeta with 0.2 threshold and accuracy as metrics. 


In [ ]:
model_1 = models.vgg16_bn
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

learn_vgg = create_cnn(data, model_1, metrics=[acc_02,f_score], model_dir='/kaggle/working')

In [ ]:
learn_vgg.lr_find()

In [ ]:
lr = 0.01
learn_vgg.fit_one_cycle(5, slice(lr))

In [ ]:
learn_vgg.save('Vgg16_bn-stage1')

In [ ]:
learn_vgg.unfreeze()
learn_vgg.lr_find()
learn_vgg.recorder.plot()

In [ ]:
learn_vgg.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn_vgg.save('Vgg16_bn-stage2')

In [ ]:
learn_vgg.recorder.plot_losses()

In [ ]:
learn_vgg.recorder.plot_metrics()

## Experiment 2

> For the first experiment: 
> + Using resnet50 (pretrained on imagenet)
> + fbeta with 0.2 threshold and accuracy as metrics. 


In [ ]:
learn_resnet.recorder.plot_losses()
learn_resnet.recorder.plot_metrics()

In [ ]:
model_2 = models.resnet50
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

learn_resnet = create_cnn(data, model_2, metrics=[acc_02,f_score], model_dir='/kaggle/working')

In [ ]:
learn_resnet.lr_find()

In [ ]:
learn_resnet.recorder.plot()

In [ ]:
lr = 0.01
learn_resnet.fit_one_cycle(5, slice(lr))

In [ ]:
learn_resnet.save('resnet50_stage1')

In [ ]:
learn_resnet.unfreeze()

In [ ]:
learn_resnet.lr_find()
learn_resnet.recorder.plot()

In [ ]:
learn_resnet.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn_resnet.save('resnet50_stage2')

In [ ]:
learn_resnet.export(file=Path("/kaggle/working/export.pkl"))

In [ ]:
learn_resnet.recorder.plot_losses()

In [ ]:
learn_resnet.recorder.plot_metrics()

# Experiment-3

* Using DenseNet121 (pretrained on imagenet)
* Similar metrics as first experiment.

In [ ]:
model_3 = models.densenet121
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

learn_dense = create_cnn(data, model_3, metrics=[acc_02, f_score], model_dir='/kaggle/working')

In [ ]:
learn_dense.lr_find()
learn_dense.recorder.plot()

In [ ]:
lr =0.01
learn_dense.fit_one_cycle(5, slice(lr))

In [ ]:
learn_dense.save('DenseNet121-stage1')

In [ ]:
learn_dense.unfreeze()
learn_dense.lr_find()
learn_dense.recorder.plot()

In [ ]:
learn_dense.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn_dense.save('DenseNet121-stage1')

In [ ]:
learn_dense.recorder.plot_losses()

In [ ]:
learn_dense.recorder.plot_metrics()

# Prediction

In [ ]:
def get_preds(obj, learn, thresh = 0.2):
    labelss = []
    # get list of classes from Learner object
    for item in learn.data.c2i:
        labelss.append(item)

    predictions = {}
    x=0
    for item in obj:
        acc= round(item.item(), 3)
        if acc > thresh:
            predictions[labelss[x]] = acc
        x+=1
        
    # sorting predictions by highest accuracy
    predictions ={k: v for k, v in sorted(predictions.items(), key=lambda item: item[1], reverse=True)}

    return predictions

In [ ]:
img1 = open_image('../input/planets-dataset/planet/planet/test-jpg/test_10034.jpg')

_,_,pred_pct = learn_dense.predict(img1)
img1.show()
print(get_preds(pred_pct,learn_dense))

In [ ]:
img2 = open_image('../input/planets-dataset/planet/planet/test-jpg/test_1005.jpg')

_,_,pred_pct = learn_dense.predict(img2)
img2.show()
print(get_preds(pred_pct,learn_dense))

In [ ]:
img3 = open_image('../input/planets-dataset/test-jpg-additional/test-jpg-additional/file_7845.jpg')

_,_,pred_pct = learn_dense.predict(img3)
img3.show()
print(get_preds(pred_pct,learn_dense))

In [ ]:
img4 = open_image('../input/planets-dataset/planet/planet/train-jpg/train_4.jpg')

_,_,pred_pct = learn_dense.predict(img4)
img4.show()
print(get_preds(pred_pct,learn_dense))

In [ ]:
img5 = open_image('../input/planets-dataset/test-jpg-additional/test-jpg-additional/file_7788.jpg')

_,_,pred_pct = learn_dense.predict(img5)
img5.show()
print(get_preds(pred_pct,learn_dense))